In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
from dateutil.rrule import rrule, DAILY

In [2]:
#чтение данных для дальнейшей обработки
sell = pd.read_csv('MS-b1-sell.csv')
inventory = pd.read_csv('MS-b1-inventory.csv')
supply = pd.read_csv('MS-b1-supply.csv')

In [3]:
#чтение данных для проверки пунктов 1 и 2
daily_check = pd.read_csv('MS-b1-daily.csv')
steal_check = pd.read_csv('MS-b1-steal.csv')

In [4]:
#предобработка датафреймов, на всякий случай удаляем пустые строки
sell.date = pd.to_datetime(sell.date)
sell.dropna()
sell.info()

supply.date = pd.to_datetime(supply.date)
supply.dropna()
supply.info()

inventory.date = pd.to_datetime(inventory.date)
inventory.dropna()
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7348354 entries, 0 to 7348353
Data columns (total 2 columns):
 #   Column   Dtype         
---  ------   -----         
 0   date     datetime64[ns]
 1   sku_num  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 112.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    240 non-null    datetime64[ns]
 1   apple   240 non-null    int64         
 2   pen     240 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 5.8 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    120 non-null    datetime64[ns]
 1   apple   120 non-null    int64         
 2   pen     120 non-null    int64         
dtypes

In [5]:
#работа с данными из MS-b1-sell.csv для получения датафрейма по виду аналогичному датафреймам supply и inventory 
#определение типа товара по sku_num по 6-му и -7-му символам
test = sell

def product_type(sku_num):
    if (sku_num[6:8] == 'pe'):
        return 'pen'
    else:
        return 'apple'
    
#добавление колонки  product_type со значением pen или apple  

test['product_type'] = test.apply(lambda x: product_type(x['sku_num']), axis = 1)

In [6]:
test.sort_values(['date', 'product_type'], inplace = True)
test.head(5)

,date,sku_num,product_type
0,2006-01-01,MS-b1-ap-48914c5b-14d2-4b20-bdaf-b2ff5d9f4f0c,apple
1,2006-01-01,MS-b1-ap-6baf7287-3e6a-4728-a3b1-8613de51eef8,apple
2,2006-01-01,MS-b1-ap-83d7b005-c7d9-4deb-93a2-a8f7606d02b5,apple
3,2006-01-01,MS-b1-ap-9099c5de-028d-4b07-97be-6f6532b8674a,apple
4,2006-01-01,MS-b1-ap-ee0ce843-a979-4798-a542-75a41d1610c3,apple


In [7]:
#подсчет количества проданного товара каждого типа за каждый день
def convert_order(sku_num):
    return 1

test['sku_num'] = test.apply(lambda x: convert_order(x['sku_num']), axis = 1)
test = test.astype({'sku_num':'int'})

sell_daily = test.groupby(['date', 'product_type'])['sku_num'].sum()
sell_daily = pd.DataFrame(sell_daily)
sell_daily.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7304 entries, (Timestamp('2006-01-01 00:00:00'), 'apple') to (Timestamp('2015-12-31 00:00:00'), 'pen')
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   sku_num  7304 non-null   int64
dtypes: int64(1)
memory usage: 107.1+ KB


In [8]:
#преобразование полученного датафрейма
sell_daily = pd.DataFrame(sell_daily.unstack(level = -1))
sell_daily.columns = sell_daily.columns.droplevel(0)
sell_daily.head(5)

product_type,apple,pen
date,,
2006-01-01,1815,156
2006-01-02,1862,143
2006-01-03,1880,171
2006-01-04,1797,153
2006-01-05,1942,133


In [9]:
supply = supply.set_index('date')
inventory = inventory.set_index('date')

In [10]:
#три датафрейма: sell_daily (ежедневное количество продаж по каждому типу товара),supply и inventory приводим к одному и тому же виду:
#индексы датафреймов - даты, колонки - 'apple' и 'pen'

sell_daily.index = [*map(lambda x: x.date(), sell_daily.index)]
supply.index = [*map(lambda x: x.date(), supply.index)]
inventory.index = [*map(lambda x: x.date(), inventory.index)]

In [11]:
my_daily = sell_daily.copy()
my_steal = inventory.copy()

In [12]:
#посчет остатка товара на складе каждый день в my_daily и количества украденного товара в месяц в my_steal
#ежедневный остаток для каждого типа товара рассчитывается так:
#запускается цикл по дням от 2006-01-01 до 2015-12-31
#для каждого дня проверяем, какое именно сегодня число. 
#если сегодня 1 или 15(дни поставок),то текущий остаток - это сумма вчерашнего отстатка и сегодняшней поставки минус сегодняшние продажи
#иначе текущий остаток - это вчерашний остаток минус сегодняшние продажи
#также необходимо в текущем состоянии склада учитывать кражи. Учет и подсчет краж идет следующим образом:
#если текущий день - конец месяца, то текущий остаток сначала считается по схеме, описанной выше
#затем в датафрейм my_steal записывается разница текущего остатка и инвертаризации за данный месяц, а потом 
#значение текущего остатка заменяем значением из таблицы ежемесячной инвентаризации

start_date = date(2006, 1, 2)
end_date = date(2015, 12, 31)
start = date(2006,1,1)

my_daily.loc[start, 'apple'] = supply.loc[start, 'apple'] - sell_daily.loc[start, 'apple']
my_daily.loc[start, 'pen'] = supply.loc[start, 'pen'] - sell_daily.loc[start, 'pen']

for today in rrule(DAILY, dtstart = start_date, until = end_date):
    
    yesterday = today - timedelta(days = 1)
    tomorrow = today + timedelta(days = 1)
    
    pd_today = today.date()
    pd_yesterday = yesterday.date()
    
    if(today.strftime("%d") == '01' or today.strftime("%d") == '15'):
        my_daily.loc[pd_today, 'apple'] = my_daily.loc[pd_yesterday, 'apple'] + supply.loc[pd_today, 'apple']
        my_daily.loc[pd_today, 'pen'] = my_daily.loc[pd_yesterday, 'pen'] + supply.loc[pd_today, 'pen']
    else:
        my_daily.loc[pd_today, 'apple'] = my_daily.loc[pd_yesterday, 'apple']
        my_daily.loc[pd_today, 'pen'] = my_daily.loc[pd_yesterday, 'pen']
        
    my_daily.loc[pd_today, 'apple'] -= sell_daily.loc[pd_today, 'apple'] 
    my_daily.loc[pd_today, 'pen'] -= sell_daily.loc[pd_today, 'pen']
    
    if (today.strftime("%m") != tomorrow.strftime("%m")):
        my_steal.loc[pd_today, 'apple'] = my_daily.loc[pd_today, 'apple'] - inventory.loc[pd_today, 'apple']
        my_steal.loc[pd_today, 'pen'] = my_daily.loc[pd_today, 'pen'] - inventory.loc[pd_today, 'pen']
        my_daily.loc[pd_today, 'apple'] = inventory.loc[pd_today, 'apple']
        my_daily.loc[pd_today, 'pen'] = inventory.loc[pd_today, 'pen']

        

    
    
    

### Ежедневное состояние склада

In [13]:
print("head(5)\n", my_daily.head(5))
print("tail(5)\n",my_daily.tail(5))

head(5)
 product_type  apple   pen
2006-01-01    33271  2574
2006-01-02    31409  2431
2006-01-03    29529  2260
2006-01-04    27732  2107
2006-01-05    25790  1974
tail(5)
 product_type    apple     pen
2015-12-27    1576773  123171
2015-12-28    1574976  123011
2015-12-29    1573136  122872
2015-12-30    1571316  122732
2015-12-31    1569425  122601


### Ежемесячное количество сворованного товара

In [14]:
print("head(5)\n",my_steal.head(5))
print("tail(5)\n",my_steal.tail(5))

head(5)
             apple  pen
2006-01-31     10   11
2006-02-28      6    6
2006-03-31      7    6
2006-04-30      6   14
2006-05-31      8    1
tail(5)
             apple  pen
2015-08-31      5    7
2015-09-30      5    7
2015-10-31      8    7
2015-11-30      6    5
2015-12-31      7    7


### steal_final - итоговый датафрейм со сворованным товаром, а daily_final - итоговый датафрейм ежедневого состояния склада.

In [15]:
steal_final = my_steal.reset_index()
daily_final = my_daily.reset_index()

In [16]:
daily_final = daily_final.rename(columns ={'index':'date'})
daily_final['date'] = pd.to_datetime(daily_final['date'])

In [17]:
steal_final = steal_final.rename(columns ={'index':'date'})
steal_final['date'] = pd.to_datetime(steal_final['date'])

In [18]:
daily_final

product_type,date,apple,pen
0,2006-01-01,33271,2574
1,2006-01-02,31409,2431
2,2006-01-03,29529,2260
3,2006-01-04,27732,2107
4,2006-01-05,25790,1974
...,...,...,...
3647,2015-12-27,1576773,123171
3648,2015-12-28,1574976,123011
3649,2015-12-29,1573136,122872
3650,2015-12-30,1571316,122732


In [19]:
steal_final

,date,apple,pen
0,2006-01-31,10,11
1,2006-02-28,6,6
2,2006-03-31,7,6
3,2006-04-30,6,14
4,2006-05-31,8,1
...,...,...,...
115,2015-08-31,5,7
116,2015-09-30,5,7
117,2015-10-31,8,7
118,2015-11-30,6,5


### создание csv-файлов на основе итоговых датафреймов

In [20]:
daily_final.to_csv('MS-b1-my_daily.csv', index = False)
steal_final.to_csv('MS-b1-my_steal.csv', index = False)

### Три худших года с точки зрения самых больших краж (по каждому из товаров)

In [23]:
print("Худшие три года с точки зрения кражи apple\n", steal_final.groupby(steal_final['date'].dt.year)['apple'].sum().sort_values().tail(3))
print("Худшие три года с точки зрения кражи pen\n", steal_final.groupby(steal_final['date'].dt.year)['pen'].sum().sort_values().tail(3))

Худшие три года с точки зрения кражи apple
 date
2012    82
2011    88
2014    90
Name: apple, dtype: int64
Худшие три года с точки зрения кражи pen
 date
2009     96
2010     96
2011    100
Name: pen, dtype: int64


### Три лучших месяца с точки зрения самых эффективных продаж (по каждому из товаров)

In [24]:
sell_daily = sell_daily.reset_index()

In [26]:
sell_daily = sell_daily.rename(columns ={'index':'date'})
sell_daily['date'] = pd.to_datetime(sell_daily['date'])

In [27]:
print("Лучшие три месяца с точки зрения продажи apple\n", sell_daily.groupby(sell_daily['date'].dt.month)['apple'].sum().sort_values().tail(3))
print("Лучшие три месяца с точки зрения продажи pen\n", sell_daily.groupby(sell_daily['date'].dt.month)['pen'].sum().sort_values().tail(3))

Лучшие три месяца с точки зрения продажи apple
 date
12    579675
7     579790
10    580211
Name: apple, dtype: int64
Лучшие три месяца с точки зрения продажи pen
 date
3     44547
8     44547
12    44549
Name: pen, dtype: int64


###